# Perplexity API

In [ ]:
# perplexity_api.py

import os
import json
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

YOUR_API_KEY = "pplx-" #os.getenv("PERPLEXITY_API_KEY")
BASE_URL = os.getenv("PERPLEXITY_BASE_URL", "https://api.perplexity.ai")
MODEL = "sonar-pro"

# Initialize the client with Perplexity base URL
client = OpenAI(api_key=YOUR_API_KEY, base_url=BASE_URL)

def fetch_articles(keyword):
    """
    Uses the Perplexity Sonar model to fetch latest articles for a given keyword.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a news aggregator AI. Return ONLY a valid JSON object with the following structure:\n"
                "{\n"
                '  "keyword": "the_search_keyword",\n'
                '  "articles": [\n'
                "    {\n"
                '      "title": "Article title",\n'
                '      "author": "Author name",\n'
                '      "source": "Source name",\n'
                '      "publishedAt": "ISO date",\n'
                '      "url": "Article URL",\n'
                '      "urlToImage": "Image URL or null",\n'
                '      "context": "Article category",\n'
                '      "summary": "Brief summary",\n'
                '      "text": "Article text"\n'
                "    }\n"
                "  ]\n"
                "}\n"
                "Do not include any explanatory text before or after the JSON."
            )
        },
        {"role": "user", "content": f"Find the latest news articles about {keyword}"}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
    )
    # Extract the text from the first choice
    text = response.choices[0].message.content
    
    # Try to find JSON content within the response
    try:
        # Look for JSON object between curly braces
        import re
        json_match = re.search(r'({[\s\S]*})', text)
        if json_match:
            json_text = json_match.group(1)
            articles = json.loads(json_text)
        else:
            articles = json.loads(text)
        return articles
    except Exception as e:
        raise Exception(f"Error parsing JSON: {e}\nResponse text: {text}")



def summarize_article(article_content):
    """
    Uses the Perplexity Sonar model to generate a summary (about 150 words) for the provided article content.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that summarizes text in approximately 150 words. "
                "Please summarize the following article content."
            )
        },
        {
            "role": "user",
            "content": article_content
        }
    ]
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
    )
    summary = response.choices[0].message.content.strip()
    return summary


if __name__ == '__mdaain__':
    # Test the article fetching functionality
    print("Testing article fetching...")
    try:
        articles = fetch_articles("artificial intelligence")
        print(f"Successfully fetched {len(articles)} articles")
        print("Sample article:")
        print(json.dumps(articles[0], indent=2))
    except Exception as e:
        print(f"Error fetching articles: {e}")

    # Test the summarization functionality
    print("\nTesting article summarization...")
    try:
        sample_text = """
        Artificial intelligence has transformed various industries, from healthcare to transportation.
        Machine learning algorithms can now diagnose diseases, drive cars, and even create art.
        The rapid advancement of AI technology has raised both excitement about its potential and
        concerns about its impact on society and the workforce.
        """
        summary = summarize_article(sample_text)
        print("Summary:")
        print(summary)
    except Exception as e:
        print(f"Error generating summary: {e}")

In [ ]:
articles = fetch_articles("artificial intelligence")

In [ ]:
import os
import json
from openai import OpenAI
# from dotenv import load_dotenv

# Load environment variables
# load_dotenv()

YOUR_API_KEY = "pplx-" #os.getenv("PERPLEXITY_API_KEY")
BASE_URL = os.getenv("PERPLEXITY_BASE_URL", "https://api.perplexity.ai")
MODEL = "sonar-pro"

# Initialize the client with Perplexity base URL
client = OpenAI(api_key=YOUR_API_KEY, base_url=BASE_URL)

In [14]:
import requests
from pydantic import BaseModel
from typing import List, Optional


class Article(BaseModel):
    title: str
    author: str
    context: str
    summary: str
    text: str


class NewsResponse(BaseModel):
    keyword: str
    articles: List[Article]


url = "https://api.perplexity.ai/chat/completions"
headers = {"Authorization": f"Bearer {YOUR_API_KEY}"}

keyword = "elon musks views on german elections"

payload = {
    "model": "sonar",
    "messages": [
        {
            "role": "system",
            "content": (
                "You are a news aggregator AI. Return ONLY a valid JSON object with the following structure:\n"
                "{\n"
                '  "keyword": "the_search_keyword",\n'
                '  "articles": [\n'
                "    {\n"
                '      "title": "Article title",\n'
                '      "author": "Author name",\n'
                '      "source": "Source name",\n'
                '      "context": "Article category",\n'
                '      "summary": "Brief summary",\n'
                '      "text": "Article text"\n'
                '      "citation": "Citation"\n'
                "    }\n"
                "  ]\n"
                "}\n"
                "Do not include any explanatory text before or after the JSON."
            ),
        },
        {"role": "user", "content": f"Find the latest 3 news articles about {keyword}"},
    ],
    # "response_format": {
        # "type": "json_schema",
        # "json_schema": {"schema": NewsResponse.model_json_schema()},
    # },
}

response = requests.post(url, headers=headers, json=payload).json()
print(response["choices"][0]["message"]["content"])


```json
{
  "keyword": "Elon Musk views on German elections",
  "articles": [
    {
      "title": "Elon Musk’s Bid to Propel Germany’s Far-Right Party to Victory Has Failed",
      "author": "Not specified",
      "source": "Mother Jones",
      "context": "Politics",
      "summary": "Elon Musk's efforts to boost the far-right Alternative for Germany (AfD) party in the German elections did not result in a victory, despite the party achieving a historic second place with about 20% of the vote.",
      "text": "Elon Musk has been actively supporting the AfD, stating that only they can save Germany. However, the party remains politically isolated as mainstream parties have ruled out forming a coalition with them.",
      "citation": "[1]"
    },
    {
      "title": "Musk congratulates far-right Weidel for coming 2nd in German election",
      "author": "Not specified",
      "source": "Politico",
      "context": "Politics",
      "summary": "Elon Musk congratulated Alice Weidel, the l

In [17]:
response['citations']

['https://www.motherjones.com/politics/2025/02/elon-musk-germany-afd-election-results/',
 'https://www.politico.eu/article/elon-musk-viktor-orban-congratulate-alice-weidel-germany-election-alternative-for-deutschland/',
 'https://leibniz-hbi.de/en/elon-musk-the-afd-and-the-agenda-setting-of-the-radical-right-in-the-2025-german-federal-election/',
 'https://euobserver.com/rule-of-law/ar72c796ad',
 'https://timesofindia.indiatimes.com/world/us/how-elon-musks-favourite-party-afd-performed-in-2025-german-election/articleshow/118532846.cms']

# Brave API

In [ ]:
brave_key="-s6xeyg1F2"

In [22]:
import requests

# Function to perform Brave News Search
# Parameters:
# - query: Search term (Required)
# - api_key: API key for authentication (Required)
# - count: Number of results (Optional, default=20, max=50)
# - country: Country code for localized results (Optional, default='US')
# - language: Search language (Optional, default='en')
# - ui_lang: User interface language (Optional, default='en-US')
# - offset: Number of results to skip for pagination (Optional, default=0, max=9)
# - spellcheck: Enable spellchecking (Optional, default=1)
# - safesearch: Level of content filtering ('off', 'moderate', 'strict', default='moderate')
# - freshness: Filter by discovery date (Optional, e.g., 'pd' for past day)
# - extra_snippets: Allow up to 5 alternative excerpts (Optional)
def brave_news_search(query, api_key, count=2, country="US", language="en", ui_lang="en-US", offset=0, spellcheck=0, safesearch="moderate", freshness='pd', extra_snippets=False):
    url = "https://api.search.brave.com/res/v1/news/search"
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": api_key
    }
    params = {
        "q": query,
        "count": count,
        "country": country,
        "search_lang": language,
        "ui_lang": ui_lang,
        "offset": offset,
        "spellcheck": spellcheck,
        "safesearch": safesearch
    }
    if freshness:
        params["freshness"] = freshness
    if extra_snippets:
        params["extra_snippets"] = 1
    
    response = requests.get(url, headers=headers, params=params)
    return response.json()

if __name__ == "__main__":
    API_KEY = brave_key  # Replace with your Brave API key
    search_query = "sebi chairman"
    
    print("Performing News Search...")
    news_results = brave_news_search(search_query, API_KEY)
    print(news_results)


Performing News Search...
{'type': 'news', 'query': {'original': 'sebi chairman', 'spellcheck_off': False, 'show_strict_warning': False}, 'results': [{'type': 'news_result', 'title': 'Govt Shortlists Candidates for Next SEBI Chairman', 'url': 'https://www.deccanchronicle.com/nation/govt-shortlists-candidates-for-next-sebi-chairman-1863636', 'description': 'Five top bureaucrats shortlisted to succeed SEBI chief Madhabi Puri Buch, with Ajay Seth and Ashwani Bhatia among frontrunners', 'age': '1 day ago', 'page_age': '2025-02-26T13:43:33', 'meta_url': {'scheme': 'https', 'netloc': 'deccanchronicle.com', 'hostname': 'www.deccanchronicle.com', 'favicon': 'https://imgs.search.brave.com/kwhSQQ0ZQKE9opCL0kctDrCh4D_YIOS2HJaj6N181Bg/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvZTQ2MjkwMTNi/NDA3ZTAwZmM4YWQ2/YTkxNWEzYWYzM2Qz/NzBjOTFlZTMzYTFj/ZWI2NDM5MzQ0N2Vh/NmE2Mzg2OS93d3cu/ZGVjY2FuY2hyb25p/Y2xlLmNvbS8', 'path': '› nation  › govt-shortlists-candidates-for-next-sebi

In [23]:
news_results

{'type': 'news',
 'query': {'original': 'sebi chairman',
  'spellcheck_off': False,
  'show_strict_warning': False},
 'results': [{'type': 'news_result',
   'title': 'Govt Shortlists Candidates for Next SEBI Chairman',
   'url': 'https://www.deccanchronicle.com/nation/govt-shortlists-candidates-for-next-sebi-chairman-1863636',
   'description': 'Five top bureaucrats shortlisted to succeed SEBI chief Madhabi Puri Buch, with Ajay Seth and Ashwani Bhatia among frontrunners',
   'age': '1 day ago',
   'page_age': '2025-02-26T13:43:33',
   'meta_url': {'scheme': 'https',
    'netloc': 'deccanchronicle.com',
    'hostname': 'www.deccanchronicle.com',
    'favicon': 'https://imgs.search.brave.com/kwhSQQ0ZQKE9opCL0kctDrCh4D_YIOS2HJaj6N181Bg/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvZTQ2MjkwMTNi/NDA3ZTAwZmM4YWQ2/YTkxNWEzYWYzM2Qz/NzBjOTFlZTMzYTFj/ZWI2NDM5MzQ0N2Vh/NmE2Mzg2OS93d3cu/ZGVjY2FuY2hyb25p/Y2xlLmNvbS8',
    'path': '› nation  › govt-shortlists-candidates